In [18]:
import sqlite3
from sqlite3 import Error
import pandas as pd
#library for generic sqlite3 functions

class PySQLite():
    def __init__(self,dbname):
        self.dbname=dbname
        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()
        conn.close()

        
    def MakeTable(self,table,column_array):
        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()
        
        Query = """
        CREATE TABLE  
        {tablename}
        {columns}
        """.format(tablename=table,columns=tuple(column_array))
    
        c.execute(Query)

        conn.commit()
        conn.close()
        
    
    def DBdescription(self,table):
        conn = sqlite3.connect(self.dbname)
        query = "SELECT * FROM {}".format(table)                
        df = pd.read_sql_query(query,conn)
        conn.close()
        return(df)
        

    def InsertRecord(self,table,record):
        
        valuestring="("
        for i in range(len(record)):
            valuestring+="?,"
              
        valuestring=valuestring[:-1]
        valuestring+=")"
        
        
        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()
        
        Query = """
        INSERT INTO
        {tablename}
        VALUES
        {values}
        """.format(tablename=table,values=valuestring)
    
        
        c.execute(Query,record)

        conn.commit()
        conn.close()        
        
    def AddField(self,table,field,f_type):    
        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()
        
        Query = """
        ALTER TABLE {tablename}
        ADD COLUMN {colname} {fieldtype} 
        """.format(tablename=table,colname=field,fieldtype=f_type)
        c.execute(Query)

        conn.commit()
        conn.close()              
    
    
    def UpdateRecord(self,name,table,updatefield,value):
        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()
        
        Query = """
        UPDATE 
            {tablename} 
        SET
            {fieldname}=(?)   
        WHERE 
           name=(?)
        """.format(tablename=table,fieldname=updatefield)
        c.execute(Query,(value,name))

        conn.commit()    
        conn.close()
        
    def GetRecord(self,table,identifier,field,value):
        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()
        
        
        value = " \"" + value + "\""
        
        
        Query = """
        SELECT  
            {fieldname} 
        FROM
            {tablename}   
        WHERE
            {identifierfield}=
           {valuename}
        """.format(fieldname=field,tablename=table,identifierfield=identifier,valuename=value)
        c.execute(Query)

        result = c.fetchone()[0]
        conn.close()
        return(result)
    
    def ExcelInput(self,excelfile,sheet,headings,table):
        # not sure why, but data gets garbled unless you put some value (say 0) in rows below main data
        df = pd.read_excel(excelfile,sheet_name=sheet)
        df2=df[headings]        

        conn = sqlite3.connect(self.dbname)
        c = conn.cursor()      
        i=0
                       
        
        for rows in df2.iterrows():
            self.InsertRecord(table,df2.iloc[i])            
            i+=1
            conn.commit()
        
        
        
        
def Midstream():
    #separate script for these initialization functions - makes tables, pulls data, etc
    DB = PySQLite("test1228v1.db")
    headings=['Name','Ticker','Revenue','RevG','VCost','FCost','EBITDA','Capex',
                  'CommonDistro','PfdDistro','AvgIntCost','TotalDebt','Cash',
                  'CrudePXSensitivity','NGPXSensitivity','CrudeVolSensitivity',
                  'NGVolSensitivity','FiveISIN','SevenISIN','TenISIN','ThirtyISIN','PerpISIN',
                  'TargetType','FiveTarget','SevenTarget','TenTarget','ThirtyTarget',
                  'PerpTarget']
#         DB.MakeTable('test_table1',headings)
    df=DB.DBdescription('test_table1')
    print(df)
    
        
#         DB.ExcelInput(r'Energy122820.xlsx','MSTemplate',headings,'test_table1')

        
#         print(headings)
#         TestDB.MakeTable('test_table13',headings)
        
#         TestDB.DBdescription('test_table13')
#         Rev0=TestDB.GetRecord('test_table13','Name','Revenue','ENLK')
#         growth=TestDB.GetRecord('test_table13','Name','RevG','ENLK')
        
#         TestDB.MakeTable("test_table2",["name","col1", "col2"])
#         TestDB.InsertRecord("test_table2",["APA",5,10])
#         TestDB.DBdescription("test_table2")
#         TestDB.AddField("test_table2","country","text")
#         TestDB.UpdateRecord("APA","test_table2","country","USA")
        
#         TestDB.InsertRecord("test_table2",["OXY",1,2,"Canada"])
#         TestDB.AddField("test_table2","type","text")
#         TestDB.UpdateRecord("OXY","test_table2","type","HY")
#         TestDB.DBdescription("test_table2")
#         print("\"oxy\"")  
#         answer=TestDB.GetRecord("test_table2","name","type","OXY")
#         print(answer)

    
    


# Midstream()

  

    Name Ticker  Revenue  RevG   VCost  FCost  EBITDA    Capex  CommonDistro  \
0    ETP     ET   9955.0   0.0  6376.0  949.0  2866.0  578.000    411.401690   
1    PAA    PAA   5833.0   0.0  5107.0  315.0   411.0  134.000    131.125788   
2    DCP    DCP   1586.0   0.0  1218.0  212.0   156.0   23.000     81.257085   
3    WES    WES    679.0   0.0   164.0   60.6   454.4   59.197    129.437049   
4   ENLK   ENLC    929.0   0.0   550.0  120.0   259.0   50.800     51.521344   
5  ONEOK    OKE   2174.0   0.0  1266.0  204.0   704.0  351.036    415.511569   
6   None   None      NaN   NaN     NaN    0.0     NaN      NaN           NaN   

    PfdDistro  ...     SevenISIN       TenISIN    ThirtyISIN      PerpISIN  \
0   83.966805  ...  US29278NAN30  US29278NAQ60  US29278NAR44  US29278NAT00   
1   45.666667  ...          None  US72650RBN17  US72650RBH49  US726503AE55   
2   34.000000  ...  US23311VAJ61  US23311VAH06  US23311VAF40  US23311PAA84   
3    0.000000  ...  US958254AF13  US958667AC17 